In [94]:
# https://www.kaggle.com/c/titanic/overview

In [160]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from catboost import CatBoostClassifier

In [161]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [138]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [139]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [162]:
for i in train['Name']:
    if 'Master.' in i:
        train['Name'] = train['Name'].replace({i:int(0)})
    elif 'Miss.' in i:
        train['Name'] = train['Name'].replace({i:int(1)})
    elif 'Mr.' in i:
        train['Name'] = train['Name'].replace({i:int(2)})
    elif 'Mrs.' in i:
        train['Name'] = train['Name'].replace({i:int(3)})
    
train['Name'] = train['Name'].astype(int)

In [163]:
for i in test['Name']:
    if 'Master.' in i:
        test['Name'] = test['Name'].replace({i:int(0)})
    elif 'Miss.' in i:
        test['Name'] = test['Name'].replace({i:int(1)})
    elif 'Mr.' in i:
        test['Name'] = test['Name'].replace({i:int(2)})
    elif 'Mrs.' in i:
        test['Name'] = test['Name'].replace({i:int(3)})
test['Name'] = test['Name'].astype(int)
# test['Name'] = pd.to_numeric(test['Name'],errors='coerce').astype(int)

In [33]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,2,male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,3,female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,1,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,3,female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,2,male,35.0,0,0,373450,8.0500,NaN,S


In [34]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,2,male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,3,female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,2,male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,2,male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,3,female,22.0,1,1,3101298,12.2875,NaN,S


In [164]:
train['Age'] = train['Age'].fillna(train['Age'].median())
train['Sex'] = train['Sex'].map({'male':1,'female':0})
train['Embarked'] = train['Embarked'].map({'S':3,'C':2,'Q':1})
# train.drop(['PassengerId','Name','Ticket','Cabin'],axis="columns",inplace=True)
train['Family'] = train['SibSp']+train['Parch']
train.drop(['PassengerId','Ticket','Cabin','SibSp','Parch'],axis="columns",inplace=True)

In [165]:
test['Age'] = test['Age'].fillna(test['Age'].median())
test['Fare'] = test['Fare'].fillna(test['Fare'].median())
test['Sex'] = test['Sex'].map({'male':1,'female':0})
test['Embarked'] = test['Embarked'].map({'S':3,'C':2,'Q':1})
# test.drop(['Name','Ticket','Cabin'],axis="columns",inplace=True)
test['Family'] = test['SibSp']+test['Parch']
test.drop(['Ticket','Cabin','SibSp','Parch'],axis="columns",inplace=True)

In [69]:
train.head()

,Survived,Pclass,Name,Sex,Age,Fare,Embarked,Family
0,0,3,2.0,1,22.0,7.2500,3,1
1,1,1,3.0,0,38.0,71.2833,2,1
2,1,3,1.0,0,26.0,7.9250,3,0
3,1,1,3.0,0,35.0,53.1000,3,1
4,0,3,2.0,1,35.0,8.0500,3,0


In [70]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,Fare,Embarked,Family
0,892,3,2.0,1,34.5,7.8292,1,0
1,893,3,3.0,0,47.0,7.0000,3,1
2,894,2,2.0,1,62.0,9.6875,1,0
3,895,3,2.0,1,27.0,8.6625,3,0
4,896,3,3.0,0,22.0,12.2875,3,2


In [166]:
X_train = train.iloc[:,1:]
Y_train = train.iloc[:,0]
X_test = test.iloc[:,1:]

In [117]:
X_train.isna().sum()

Pclass      0
Name        0
Sex         0
Age         0
Fare        0
Embarked    0
Family      0
dtype: int64

In [118]:
X_test.isna().sum()

Pclass      0
Name        0
Sex         0
Age         0
Fare        0
Embarked    0
Family      0
dtype: int64

In [119]:
X_train.head()

,Pclass,Name,Sex,Age,Fare,Embarked,Family
0,3,2,1,22.0,7.2500,3,1
1,1,3,0,38.0,71.2833,2,1
2,3,1,0,26.0,7.9250,3,0
3,1,3,0,35.0,53.1000,3,1
4,3,2,1,35.0,8.0500,3,0


In [120]:
X_test.head()

,Pclass,Name,Sex,Age,Fare,Embarked,Family
0,3,2,1,34.5,7.8292,1,0
1,3,3,0,47.0,7.0000,3,1
2,2,2,1,62.0,9.6875,1,0
3,3,2,1,27.0,8.6625,3,0
4,3,3,0,22.0,12.2875,3,2


In [135]:
X_train.dtypes

Pclass        int64
Name          int32
Sex           int64
Age         float64
Fare        float64
Embarked      int64
Family        int64
dtype: object

In [136]:
X_test.dtypes

Pclass        int64
Name          int32
Sex           int64
Age         float64
Fare        float64
Embarked      int64
Family        int64
dtype: object

In [167]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0,1,2,5])], remainder='passthrough')
X_train = ct.fit_transform(X_train)

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0,1,2,5])], remainder='passthrough')
X_test = ct.fit_transform(X_test)

In [168]:
X_train.shape

(891, 15)

In [169]:
X_train[0]

array([ 0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,  0.  ,  0.  ,  1.  ,
        0.  ,  0.  ,  1.  , 22.  ,  7.25,  1.  ])

In [170]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
ct = ColumnTransformer([('xyz',StandardScaler(),[12,13])], remainder='passthrough')
X_train = ct.fit_transform(X_train)
ct = ColumnTransformer([('xyz',StandardScaler(),[12,13])], remainder='passthrough')
X_test = ct.fit_transform(X_test)

In [171]:
# print(X_train)
# print(Y_train)
from sklearn.model_selection import train_test_split
data_tr, data_te, result_tr, result_te = train_test_split(X_train, Y_train, test_size=0.2, random_state=0)

In [172]:
np.shape(X_train)

(891, 15)

In [173]:
np.shape(X_test)

(418, 15)

In [177]:
acc = []
# for i in range(1,500):
model = CatBoostClassifier(iterations=204,depth=6)  
model.fit(data_tr, result_tr)
        # preds_valid = model.predict(data_tr)

        # print(f1_score(y_train, preds_valid, average='weighted'))
y_pred = model.predict(data_te)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(result_te, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
#     acc.append(accuracy*100.0)
    
# f1_score(y_train, predictions, average='weighted')

Learning rate set to 0.038284
0:	learn: 0.6652600	total: 2.82ms	remaining: 573ms
1:	learn: 0.6437541	total: 5.55ms	remaining: 561ms
2:	learn: 0.6222277	total: 8.41ms	remaining: 564ms
3:	learn: 0.6027327	total: 11.8ms	remaining: 590ms
4:	learn: 0.5849372	total: 14.5ms	remaining: 578ms
5:	learn: 0.5707908	total: 16.6ms	remaining: 546ms
6:	learn: 0.5587201	total: 18.3ms	remaining: 516ms
7:	learn: 0.5460280	total: 21.1ms	remaining: 518ms
8:	learn: 0.5334596	total: 24.6ms	remaining: 532ms
9:	learn: 0.5239839	total: 27.7ms	remaining: 537ms
10:	learn: 0.5130538	total: 31ms	remaining: 543ms
11:	learn: 0.5028466	total: 33.9ms	remaining: 543ms
12:	learn: 0.4937921	total: 37.3ms	remaining: 548ms
13:	learn: 0.4872158	total: 41ms	remaining: 556ms
14:	learn: 0.4789693	total: 44.3ms	remaining: 558ms
15:	learn: 0.4721307	total: 47.7ms	remaining: 561ms
16:	learn: 0.4659283	total: 51ms	remaining: 561ms
17:	learn: 0.4595877	total: 54.2ms	remaining: 560ms
18:	learn: 0.4538123	total: 57.5ms	remaining: 560m

199:	learn: 0.2897884	total: 664ms	remaining: 13.3ms
200:	learn: 0.2892450	total: 667ms	remaining: 9.95ms
201:	learn: 0.2890354	total: 670ms	remaining: 6.63ms
202:	learn: 0.2886401	total: 673ms	remaining: 3.31ms
203:	learn: 0.2875978	total: 676ms	remaining: 0us
Accuracy: 86.59%


In [175]:
print(max(acc))

86.59217877094973


In [176]:
pos = max(enumerate(acc), key=lambda x: x[1])[0]
print(pos)

203


In [53]:
print(acc[pos])

85.47486033519553


In [178]:
model = CatBoostClassifier(iterations=204,depth=6)
model.fit(X_train, Y_train)
# preds_valid = model.predict(data_tr)

# print(f1_score(y_train, preds_valid, average='weighted'))
y_pred = model.predict(X_test)
pred = []
pred = [round(value) for value in y_pred]

# accuracy = accuracy_score(result_te, predictions)
# print("Accuracy: %.2f%%" % (accuracy * 100.0))

Learning rate set to 0.042131
0:	learn: 0.6606923	total: 3.44ms	remaining: 699ms
1:	learn: 0.6375166	total: 7.11ms	remaining: 718ms
2:	learn: 0.6164801	total: 8.89ms	remaining: 596ms
3:	learn: 0.5992500	total: 11.9ms	remaining: 594ms
4:	learn: 0.5782527	total: 15.5ms	remaining: 615ms
5:	learn: 0.5584385	total: 18.6ms	remaining: 615ms
6:	learn: 0.5432254	total: 21.8ms	remaining: 615ms
7:	learn: 0.5302658	total: 25.2ms	remaining: 616ms
8:	learn: 0.5156374	total: 28.5ms	remaining: 619ms
9:	learn: 0.5024951	total: 32.1ms	remaining: 622ms
10:	learn: 0.4912803	total: 35.3ms	remaining: 620ms
11:	learn: 0.4807017	total: 38.8ms	remaining: 621ms
12:	learn: 0.4713486	total: 43.2ms	remaining: 635ms
13:	learn: 0.4634951	total: 47.8ms	remaining: 648ms
14:	learn: 0.4569223	total: 52.2ms	remaining: 658ms
15:	learn: 0.4487467	total: 55.5ms	remaining: 652ms
16:	learn: 0.4415777	total: 58.7ms	remaining: 646ms
17:	learn: 0.4369201	total: 63.1ms	remaining: 652ms
18:	learn: 0.4319424	total: 66ms	remaining: 

163:	learn: 0.3027951	total: 496ms	remaining: 121ms
164:	learn: 0.3024456	total: 499ms	remaining: 118ms
165:	learn: 0.3013393	total: 502ms	remaining: 115ms
166:	learn: 0.3002213	total: 505ms	remaining: 112ms
167:	learn: 0.2999656	total: 508ms	remaining: 109ms
168:	learn: 0.2994389	total: 512ms	remaining: 106ms
169:	learn: 0.2990042	total: 515ms	remaining: 103ms
170:	learn: 0.2985427	total: 519ms	remaining: 100ms
171:	learn: 0.2979139	total: 523ms	remaining: 97.2ms
172:	learn: 0.2973280	total: 526ms	remaining: 94.3ms
173:	learn: 0.2964611	total: 529ms	remaining: 91.2ms
174:	learn: 0.2961571	total: 532ms	remaining: 88.2ms
175:	learn: 0.2959192	total: 535ms	remaining: 85.1ms
176:	learn: 0.2958660	total: 537ms	remaining: 81.9ms
177:	learn: 0.2954469	total: 540ms	remaining: 78.8ms
178:	learn: 0.2943744	total: 542ms	remaining: 75.8ms
179:	learn: 0.2939530	total: 545ms	remaining: 72.7ms
180:	learn: 0.2925940	total: 549ms	remaining: 69.7ms
181:	learn: 0.2923743	total: 551ms	remaining: 66.7ms
1

In [179]:
pd.DataFrame(pred).to_csv('sol.csv')

In [180]:
print(len(pred))

418
